# Trigonometric Integrals and Orthogonal Functions
The following are $\sin(x)$, $\cos(x)$, and $\sin^3(x)$ on the interval $x\in (-\pi, \pi)$.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import scipy as sp 

x = sp.linspace(start=-sp.pi, stop=sp.pi, num=100000)
# plot sin(x)
plt.subplot(1, 3, 1)
plt.fill(x, sp.sin(x), "b")
plt.grid(True)
plt.xlabel(r'$x$')
plt.ylabel(r'$\sin(x)$')
# plot cos(x)
plt.subplot(1, 3, 2)
plt.fill(x, sp.cos(x), "b")
plt.grid(True)
plt.xlabel(r'$x$')
plt.ylabel(r'$\cos(x)$')
# plot sin^3(x)
plt.subplot(1, 3, 3)
plt.fill(x, sp.sin(x)**3, "b")
plt.grid(True)
plt.xlabel(r'$x$')
plt.ylabel(r'$\sin^3(x)$')
plt.tight_layout()
plt.show()

## Riemann Sum, or, Integrating Numerically
Computers, broadly speaking, cannot do algebra. So, how do we approximate difficult (think many dimensions) or even impossible integrals _in silica_ ? In the way that Riemann originally proposed:
$$I = \int_a^b f(x)\text{d}x = \lim_{n\rightarrow\infty}\sum_{i=1}^{n}f(t_i)\underbrace{(x_i - x_{i-1})}_{\Delta_i}$$
I.e. divide the range of integration, $[a, b]$, into $n$ sub-intervals, ["tagging" a point](https://en.wikipedia.org/wiki/Integral#Riemann_integral), $t_i$ for every subinterval. Then, for that $n$, add up the function's value at $t_i$ multiplied by the width of the interval containing $t_i$. Finally, the limit of $n$ going to infinity is the *exact* value of the integral; however, there by definition isn't enough silicon in existence to store and compute $n$ large enough to approach infinity, so we choose $n$ that is computationally feasible.

Almost always, the partition $$a=x_0\leq t_1 \leq x_1 \leq x_2 \leq \cdots \leq x_{n-1} \leq t_n \leq x_n = b$$ evenly partitions the range $[a, b]$ by taking $\frac{b-a}{n}$, so $\Delta_i = \Delta \ \forall i$. Also, $t_i$ is commonly taken as the left-endpoint of each subinterval (known as [Left Riemann summation](https://en.wikipedia.org/wiki/Riemann_sum#Methods)), so the expression for $I$ becomes: $$I = \lim_{n\rightarrow\infty}\Delta\sum_{i=1}^{n}f(x_{i-1})$$
### Approximate $I_{1,1}$
$\sin(mx)$ and $\cos(nx)$ [are orthogonal](https://en.wikipedia.org/wiki/Orthogonal_functions) $\forall m,n \in \mathbb{Z}$. So, $\langle\sin(mx), \cos(nx)\rangle=0 \ \forall m,n \in \mathbb{Z}$. Let's see how close we can get to 0 numerically evaluating $I_{1,1}$, $$I_{1,1} = \int_{-\pi}^{\pi}\sin(x)\cos(x)\text{d}x = \lim_{n\rightarrow\infty}\sum_{i=1}^{n}\sin(t_i)\cos(t_i)\Delta_i = 0$$
### How Small to Make $\Delta$?
There is a fundamentally small unit of precision in computing called $\epsilon$, or machine epsilon. Its definition is the smallest floating-point number such that $1 + \epsilon \neq 1$. On this 64-bit machine, it is

In [ ]:
print(sp.finfo(sp.pi).eps)

How small to make $\Delta$ is the same question as "how many sub-intervals, $n$, to make?". This is a question of how accurate we want to be; we can't get to 0 exactly, but we can get to within machine epsilon. Is 16 decimal places overkill, though? How about just using the endpoints, plus the integers between $-\pi$ and $\pi$, namely: $[-\pi, -3, -2, -1, 0, 1, 2, 3, \pi]$? 

In [ ]:
# Left Riemann sum with P = [a, b, and integers in between]
x = [-sp.pi, -3, -2, -1, 0, 1, 2, 3, sp.pi]
partition = [[-sp.pi, -3], [-3, -2], [-2, -1], [-1, 0], 
             [0,1], [1, 2], [2, 3], [3, sp.pi]]
t_i = [subinterval[0] for subinterval in partition]
f_of_t_i = sp.sin(t_i) * sp.cos(t_i)
print(f_of_t_i)
Delta_i = [subinterval[1] - subinterval[0] for subinterval in partition]
print(Delta_i)
I_11_approximation = sum(f_of_t_i * Delta_i)
print(f"\nApproximation of integral: {I_11_approximation}")

In [ ]:
# Now, Left Riemann sum with n=100 and Delta = (b-a)/n
ti, Delta = sp.linspace(-sp.pi, sp.pi, num=100, retstep=True, endpoint=False)
f_of_ti = sp.sin(ti) * sp.cos(ti)
I_11_approximation = Delta * f_of_t_i.sum()
print(f"\nApproximation of integral: {I_11_approximation}")

In [ ]:
%matplotlib notebook
fine_x = sp.linspace(start=-sp.pi, stop=sp.pi, num=100000)
# plot sin(x)
plt.plot(fine_x, sp.sin(fine_x)*sp.cos(fine_x), color='k')
plt.grid(True)
plt.xlabel(r'$x$')
plt.ylabel(r'$\sin(x)\cos(x)$')
# add integer samples
plt.plot(t_i, f_of_t_i, marker='o', color='red', linestyle='none')
# add fine points sampled
plt.plot(ti, f_of_ti, marker='o', color='blue', linestyle='none', alpha=0.3)
plt.title(r"$f(x)=\sin(x)\cos(x)$ Sampled between $-\pi$ and $\pi$")
plt.show()

## Symbolic Integration
The symbolic Python library `sympy` will calculate $\langle \sin(mx), \cos(nx)\rangle$ "exactly" via the `sympy.integrate` function, operating on an appropriately-defined `sympy.Symbol` object.

In [ ]:
from sympy import *
init_printing(use_unicode=False, wrap_line=False, no_global=True)
y = Symbol('y')
y**2

In [ ]:
def integrate_sin_mx_cos_nx(x, m, n, a, b):
    """Given a sympy.Symbol, and four integers, a, b, m, and n, return the
    integral of sin(mx)*cos(nx)dx evaluated from a to b"""
    x = Symbol('x')
    f = sin(m*x)
    g = cos(n*x)
    I = integrate(f*g, (x, a, b))
    return I

print(integrate_sin_mx_cos_nx(x=x, m=1, n=1, a=-sp.pi, b=-sp.pi))